# Intervalos de Confianza para Proporciones
En esta libreta, se implementan funciones en Python para calcular intervalos de confianza aproximados y exactos para proporciones muestrales. Se presentan resultados tabulares y gráficos para facilitar la interpretación.


In [ ]:

import pandas as pd
from scipy.stats import norm, beta
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('seaborn-v0_8-whitegrid')

def intervalo_proporcion_wilson(x: int, n: int, confidence: float) -> dict:
    """Intervalo de confianza aproximado (Wilson) para proporciones."""
    p_hat = x / n                                                                   # estimador puntual de la proporción donde x es el número de éxitos y n el tamaño de la muestra                 
    z = norm.ppf(1 - (1 - confidence) / 2)                                          # valor crítico z
    denominator = 1 + (z**2) / n                                                    # denominador común
    center = (p_hat + (z**2) / (2 * n)) / denominator                               # centro del intervalo
    margin = (z * ((p_hat * (1 - p_hat) + (z**2) / (4 * n)) / n)**0.5) / denominator# margen de error
    se = np.sqrt(p_hat * (1 - p_hat) / n)                                           # error estándar
    lower = max(0.0, center - margin)                                               # límite inferior Wilson
    upper = min(1.0, center + margin)                                               # límite superior Wilson
    error = upper - p_hat
    return {
        "p_hat": p_hat,          # estimador puntual
        "z": z,                  # valor crítico z
        "se": se,                # error estándar
        "lower": lower,          # límite inferior Wilson
        "upper": upper,          # límite superior Wilson
        "error": error,          # error máximo
        "confidence": confidence,# nivel de confianza
        "n": n,                  # tamaño de la muestra
        "x": x                   # número de éxitos
    }

def intervalo_proporcion_clopper_pearson(x: int, n: int, confidence: float) -> tuple[float, float]:
    """Intervalo exacto (Clopper-Pearson) útil como referencia para muestras pequeñas."""
    p_hat = x / n                                                   # estimador puntual de la proporción donde x es el número de éxitos y n el tamaño de la muestra
    alpha = 1 - confidence                                          # nivel de significancia
    z = norm.ppf(1 - alpha / 2)                                     # valor crítico z
    se = np.sqrt(p_hat * (1 - p_hat) / n)                           # error estándar
    lower = beta.ppf(alpha / 2, x, n - x + 1) if x > 0 else 0.0     # límite inferior donde x es el número de éxitos y n el tamaño de la muestra
    upper = beta.ppf(1 - alpha / 2, x + 1, n - x) if x < n else 1.0 # límite superior
    error = upper - p_hat                                           # error máximo
    return {
        "p_hat": p_hat,          # estimador puntual
        "z": z,                  # valor crítico z
        "se": se,                # error estándar
        "lower": lower,          # límite inferior Clopper-Pearson
        "upper": upper,          # límite superior Clopper-Pearson
        "error": error,          # error máximo
        "confidence": confidence,# nivel de confianza
        "n": n,                  # tamaño de la muestra
        "x": x                   # número de éxitos
    }

def intervalo_proporcion_normal(x: int, n: int, confidence: float) -> dict:
    """Calcula intervalo de confianza normal aproximado para una proporción."""
    p_hat = x / n                         # estimador puntual de la proporción donde x es el número de éxitos y n el tamaño de la muestra
    alpha = 1 - confidence                # nivel de significancia
    z = norm.ppf(1 - alpha / 2)           # valor crítico z
    se = np.sqrt(p_hat * (1 - p_hat) / n) # error estándar
    error = z * se                        # error máximo
    return {
        "p_hat": p_hat,                   # estimador puntual
        "z": z,                           # valor crítico z
        "se": se,                         # error estándar
        "lower": max(0.0, p_hat - error), # límite inferior
        "upper": min(1.0, p_hat + error), # límite superior
        "error": error,                   # error máximo
        "confidence": confidence,         # nivel de confianza
        "n": n,                           # tamaño de la muestra
        "x": x                            # número de éxitos
    }   

def resumen_intervalos_proporcion(x: int, n: int, confidence: float) -> pd.DataFrame:
    """Genera un resumen tabular de los intervalos de confianza para proporciones."""
    p_hat = x / n
    wilson_lower, wilson_upper = intervalo_proporcion_wilson(x, n, confidence)
    clopper_lower, clopper_upper = intervalo_proporcion_clopper_pearson(x, n, confidence)
    data = {
        'Estimación Puntual': [p_hat],
        'Wilson Lower': [wilson_lower],
        'Wilson Upper': [wilson_upper],
        'Clopper-Pearson Lower': [clopper_lower],
        'Clopper-Pearson Upper': [clopper_upper]
    }
    return pd.DataFrame(data)